# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [84]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [146]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


bio_template = """你是一位很棒的生物家。你擅长回答生物问题。
之所以如此出色，是因为仅使用英文来回答问题。


这是一个问题：
{input}"""


cs_template = """你是一位很棒的计算机领域人员。你擅长回答计算机领域相关问题。
之所以如此出色，是因为你能够通过伪代码举例说明解释问题。

这是一个问题：
{input}"""


cn_template = """你是一位很棒的语文学家。你擅长回答语文问题。
之所以如此出色，是因为你仅使用文言文解释问题。

这是一个问题：
{input}"""

In [147]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": bio_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": cs_template,
    },
    {
        "name": "语文老师",
        "description": "适用于回答语文问题",
        "prompt_template": cn_template,
    },
]

In [148]:
llm = OpenAI(api_key='sk-9p2vA8PvWogRQ9p500783cD35b9a4815A4Af10661d69B4F6',base_url='https://api.bltcy.ai/v1',model_name="gpt-3.5-turbo-instruct")

In [149]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [150]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [151]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [152]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [153]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '语文老师: 适用于回答语文问题']


In [154]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
语文老师: 适用于回答语文问题


In [155]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [156]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
语文老师: 适用于回答语文问题

<< INPUT >>
{input}

<< OU

In [157]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [158]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？'}
> Finished chain.
{'input': '黑体辐射是什么？', 'text': '\n\n黑体辐射是指一个理想的物体在一定温度下发出的电磁辐射。根据普朗克定律，黑体辐射的能量与温度呈正比，而根据维恩位移定律，辐射的波长与温度呈反比。黑体辐射是研究热辐射和光学性质的重要模型，也是量子力学发展的重要基础。'}


In [159]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n第一个大于40的质数是41，41加一等于42，42除以3余0，所以答案是41。 '}


In [160]:
router_chain.verbose = True

In [161]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n\n黑洞是一种极其密集的天体，它的引力非常强大，甚至连光都无法逃脱。它们形成于宇宙中最大的恒星死亡时，其质量超过太阳几倍甚至上百倍。黑洞的存在是基于爱因斯坦的广义相对论理论，它们在宇宙中发挥着重要的作用，例如控制星系的演化和产生强烈的引力波。但对于黑洞内部的情况，我们目前仍不完全了解，因为它们的引力是如此强大，甚至连光都无法逃脱，所以无法直接观测到它们的内部情况。希望我的回答能帮助您更好地理解黑洞。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [162]:
print(chain.invoke("冒泡排序是什么？"))
print(chain.invoke("低头思故乡是什么意思？"))
print(chain.invoke("线粒体有什么作用？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': 'What is bubble sorting?'}
> Finished chain.
{'input': 'What is bubble sorting?', 'text': '\n\n回答：\n冒泡排序是一种简单的排序算法，它通过比较相邻的元素并交换位置来实现排序。它的基本思想是从数组的第一个元素开始，逐个比较相邻的两个元素，如果前一个元素大于后一个元素，则交换它们的位置。这样一轮比较下来，最大的元素就会被交换到数组的最后一个位置。然后再从第一个元素开始重复这个过程，直到所有元素都被排序。下面是用伪代码表示的冒泡排序算法：\n\nfor i from 0 to n-1:\n    for j from 0 to n-1-i:\n        if arr[j] > arr[j+1]:\n            swap(arr[j], arr[j+1])\n\n其中，n为数组的长度，arr为待排序的数组。通过两层循环，每次比较相邻的两个元素，并根据大小交换它们'}


> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
语文老师: {'input': '低头思故乡是什么意思？'}
> Finished chain.
{'input': '低头思故乡是什么意思？', 'text': '\n\n回答：\n低头思故乡是指在远离家乡的时候，想起家乡的情景和亲人，产生思乡之情。它表达了对家乡的眷恋和对离乡生活的不适应。这个词语出自唐代诗人孟郊的诗作《游子吟》：“家书抵万金，但寄与骄儿孤，低头弄青梅，却把旧情怜。思量半年馀，想像红衰蕉，日日趋庭户，空对镜中痴。”意思是家书虽然珍贵，但寄给远方的孤儿，低头玩弄青梅，却怀念旧日的情景。这首诗表达了游子对家乡的思念和对'}


> Entering new MultiPromptChain chain...


> Entering new LLM